In [ ]:

#import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import chardet
from datetime import datetime, timedelta
import locale
from sklearn.decomposition import PCA
from sklearn import preprocessing
import prince

#pd.set_option("future.no_silent_downcasting", True)

## Population

Fetching file for population and preview

In [ ]:
populationDF = pd.read_csv(r"DataCSV\Populationen.csv", sep = ';')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
populationDF

Column names then remove unwanted columns

In [ ]:
populationDF.columns.to_list()

Create dictionary for column names - 'old name' : 'new name'

In [ ]:
populationDFColumnsDic = {'CPR'                             : 'CPR',
                          'Alder på operationstidspunkt'    : 'Age',
                          'Køn'                             : 'Sex',
                          'Dødsdato'                        : 'DeathDate',
                          'Død inden for 1 år af operation' : 'DeathWithinYear',
                          'Operationsdato'                  : 'OperationDate',
                          'Operation SKS kode'              : 'SKSCode',
                          'Operationsnavn'                  : 'OperationName',
                          'Indlæggelsestidspunkt'           : 'HospitalizationDate',
                          'Udskrivelsestidspunkt'           : 'DischargeDate',
                          'Indlæggelsesvarrighed i dage'    : 'DurationInDays'}

In [ ]:
populationDF = populationDF.rename(columns = populationDFColumnsDic)
populationDF

In [ ]:
populationDF = populationDF.drop(columns = ['OperationName'])

## PATS

In [ ]:
PATSdf = pd.read_csv(r"DataCSV\PATS data 2018-23  pr. 150424.csv", sep = ';')


PATSdfColumnsDic = {'CPR'                      : 'CPR',
                    'Proceduredato'            : 'ProcedureDate',
                    'Alder ved operationen'    : 'Age',
                    'Procedurekode 1'          : 'SKSCode1',
                    'Procedurekode 2'          : 'SKSCode2',
                    'Procedurekode 3'          : 'SKSCode3',
                    'Procedurekode 4'          : 'SKSCode4',
                    'Procedurekode 6'          : 'SKSCode6',
                    'euroSCORE II'             : 'euroSCOREII',
                    'Dato for mors'            : 'DeathDate'}

PATSdf = PATSdf.rename(columns = PATSdfColumnsDic)
PATSdf

In [ ]:
len(list(set(PATSdf['CPR'])))

In [ ]:
PATSdf = PATSdf[PATSdf['Age'] > 17].reset_index(drop = True)
len(list(set(PATSdf['CPR'])))

In [ ]:
PATSdf = PATSdf.drop_duplicates(subset=['CPR']).reset_index()

### Cleaning up SKS codes

Check to see if multiple codes appear in a variable

In [ ]:
def sks_cleaner(SKSCode1, SKSCode2, SKSCode3, SKSCode4, SKSCode5,i) :
    while isinstance(PATSdf[SKSCode1][i], str) and len(PATSdf[SKSCode1][i]) > 5 :
        if pd.isna(PATSdf[SKSCode2][i]) :
            PATSdf.loc[i, SKSCode2] = PATSdf[SKSCode1][i][:5]
        elif pd.isna(PATSdf[SKSCode3][i]) :
            PATSdf.loc[i, SKSCode3] = PATSdf[SKSCode1][i][:5]
        elif pd.isna(PATSdf[SKSCode4][i]) :
            PATSdf.loc[i, SKSCode4] = PATSdf[SKSCode1][i][:5]
        elif pd.isna(PATSdf[SKSCode5][i]) :
            PATSdf.loc[i, SKSCode5] = PATSdf[SKSCode1][i][:5]
        PATSdf.loc[i, SKSCode1] = PATSdf[SKSCode1][i][6:]


for i in range(len(PATSdf)) :
    sks_cleaner('SKSCode1', 'SKSCode2', 'SKSCode3', 'SKSCode4', 'SKSCode6', i)
    sks_cleaner('SKSCode2', 'SKSCode1', 'SKSCode3', 'SKSCode4', 'SKSCode6', i)
    sks_cleaner('SKSCode3', 'SKSCode2', 'SKSCode1', 'SKSCode4', 'SKSCode6', i)
    sks_cleaner('SKSCode4', 'SKSCode2', 'SKSCode3', 'SKSCode1', 'SKSCode6', i)
    sks_cleaner('SKSCode6', 'SKSCode2', 'SKSCode3', 'SKSCode4', 'SKSCode1', i)

#viewMeasurements['Count'][i] = preOpConferenceDF['Measurement'].value_counts()[viewMeasurements['Measurement'][i]]


### Changing time strings to time objects

In [ ]:
dateFormat = r'%d-%b-%y'
locale.setlocale(locale.LC_ALL, 'da_DK')
for i in range(len(PATSdf)) :
    PATSdf.loc[i, 'ProcedureDate'] = datetime.strptime(PATSdf['ProcedureDate'][i], dateFormat)

In [ ]:
PATSdf

In [ ]:
PATSdf.count()

## Grouping SKS codes in PATS

In [ ]:
PATSdf['SKSCode1'] = PATSdf['SKSCode1'].str[:2]
PATSdf['SKSCode2'] = PATSdf['SKSCode2'].str[:2]
PATSdf['SKSCode3'] = PATSdf['SKSCode3'].str[:2]
PATSdf['SKSCode4'] = PATSdf['SKSCode4'].str[:2]
PATSdf['SKSCode6'] = PATSdf['SKSCode6'].str[:2]

In [ ]:
PATSsksCount = [PATSdf['SKSCode1'].value_counts(), PATSdf['SKSCode2'].value_counts(), PATSdf['SKSCode3'].value_counts(), PATSdf['SKSCode4'].value_counts(), PATSdf['SKSCode6'].value_counts()]
PATSsksCountDF = pd.DataFrame(PATSsksCount)
PATSsksCountDF = PATSsksCountDF.fillna(0).astype(int)
PATSsksCountDF.to_csv(r'FilteredData\SKSCodesInPATS.csv')

## Cleaning population

### Remove those who don't appear in the PATS data set

Steps:

Remove the ones who aren't in PATS 

Remove children 

Remove irrelevant procedures

In [ ]:
#PATScpr =  PATSdf['CPR'].to_list()

populationDF = populationDF[populationDF['CPR'].isin(PATSdf['CPR'])]

populationDF

### Remove the children

In [ ]:
populationDF = populationDF[populationDF['Age'] > 17].reset_index(drop = True)
populationDF

### Check number of unique patients

In [ ]:
#PopulationCPR =  populationDF['CPR'].to_list
len(list(set(populationDF['CPR'])))

In [ ]:
len(list(set(populationDF['CPR'])))

In [ ]:
populationDF.count()

### Procedure filter

In [ ]:
SKSFilter = pd.read_csv("Procedurekoder_filter_V3.csv", sep = ';')

SKSFilter = SKSFilter.drop(columns = ['Kolonne1', 'cardiac_surgery1'])

SKSfilteredOut = SKSFilter[SKSFilter['cardiac_surgery2'] == 'n']
SKSFilter = SKSFilter[SKSFilter['cardiac_surgery2'] == 'y']
SKSFilter['PATSsks'] = SKSFilter['operation_sks_kode'].str[1:]
SKSFilter['PATSsksSuperGroup'] = SKSFilter['operation_sks_kode'].str[1:3]

SKSFilter

In [ ]:
SKSFilter

In [ ]:
SKSfilteredOut

In [ ]:
PATSdfFiltered = PATSdf[PATSdf['SKSCode1'].isin(SKSFilter['PATSsksSuperGroup']) | PATSdf['SKSCode2'].isin(SKSFilter['PATSsksSuperGroup']) | PATSdf['SKSCode3'].isin(SKSFilter['PATSsksSuperGroup']) | PATSdf['SKSCode4'].isin(SKSFilter['PATSsksSuperGroup']) | PATSdf['SKSCode6'].isin(SKSFilter['PATSsksSuperGroup'])].reset_index(drop = True)#PATSdf['SKSCode1'].isin(another_df['AnotherColumn']) | df['Column'].isin(yet_another_df['YetAnotherColumn'])
len(list(set(PATSdfFiltered['CPR'])))

Filtering from PATSFiltered

In [ ]:
populationDFPATSFiltered = PATSdfFiltered[~PATSdfFiltered['CPR'].isin(populationDF['CPR'])].reset_index(drop = True)
viewCPR = list(set(populationDFPATSFiltered['CPR']))
viewCPRdf= pd.DataFrame(viewCPR)
viewCPRdf.to_csv(r'FilteredData\MissingInPopulation.csv', index = False)

#### Filtering out the procedures that are not relevant

In [ ]:
populationDF = populationDF[populationDF['SKSCode'].isin(SKSFilter['operation_sks_kode'])].reset_index(drop = True)

populationDF

In [ ]:
len(list(set(populationDF['CPR'])))

## Anesthesia event dataset

In [ ]:
anesthesiaEventDF = pd.read_csv(r"DataCSV\Anæstesihændelse.csv", sep = ';')

anesthesiaEventDFColumnsDic = {'CPR'                 : 'CPR',
                               'Hændelse'            : 'Event',
                               'Hændelsestidspunkt'  : 'EventTime'}

anesthesiaEventDF = anesthesiaEventDF.rename(columns = anesthesiaEventDFColumnsDic)

anesthesiaEventDF = anesthesiaEventDF[anesthesiaEventDF['CPR'].isin(populationDF['CPR'])].reset_index(drop = True)

anesthesiaEventDF

In [ ]:
len(list(set(anesthesiaEventDF['CPR'])))

In [ ]:
extraPerson = set(populationDF['CPR']) - set(anesthesiaEventDF['CPR'])
extraPerson

In [ ]:
anesthesiaEventDF['Event'].value_counts()

In [ ]:
anesthesiaEventDF.loc[2]

### Find the time of each operation

Converting the EventTime strings to time objects and then calculate the time of each procedure of each patient from these

In [ ]:
cleanAnesthesiaEventDF = pd.read_csv(r"Clean_anesthesia_times_Theis.csv", sep = ';')

#cleanAnesthesiaEventDF = cleanAnesthesiaEventDF.drop(columns = ['x1' , 'proceduredato' , 'alder_ved_operationen'])

renameDic = {                  'cpr'             : 'CPR',
                               'event'           : 'Event',
                               'event_time'      : 'EventTime',
                               'procedure_date'  : 'ProcedureDate'}

cleanAnesthesiaEventDF = cleanAnesthesiaEventDF.rename(columns = renameDic)
cleanAnesthesiaEventDF = cleanAnesthesiaEventDF.sort_values('CPR')

cleanAnesthesiaEventDF = cleanAnesthesiaEventDF[cleanAnesthesiaEventDF['CPR'].isin(populationDF['CPR'])].reset_index(drop = True)

cleanAnesthesiaEventDF

In [ ]:
dateFormat1 = r'%Y-%m-%dT%H:%M:%SZ'
cleanAnesthesiaEventDF['EventTime'] = pd.to_datetime(cleanAnesthesiaEventDF['EventTime'], format = dateFormat1)

In [ ]:
firstEventList, lastEventList, startTimeList, endTimeList, timeList = [], [], [], [], []
endPriority = ['Stop Data Indsamling' , 'CV Bypass slut' , 'Aorta tang af' , 'Aorta tang på' , 'CV Bypass Start' , 'Induktion']

cleanAnesthesiaEventDF = cleanAnesthesiaEventDF.dropna(subset=['CPR'])
for cpr, group in cleanAnesthesiaEventDF.groupby('CPR') :
    firstProcedure, lastProcedure, endTime = None, None, None
    startTime = group.loc[(group['CPR'] == cpr) & (group['Event'] == 'Induktion'), 'EventTime'].iloc[0]
    firstProcedure = group.loc[(group['CPR'] == cpr) & (group['Event'] == 'Induktion'), 'Event'].iloc[0]

    for lastEvent in endPriority :
        lastEvents = group.loc[group['Event'] == lastEvent, 'EventTime']
        if not lastEvents.empty :
            endTime = lastEvents.iloc[0]
            lastProcedure = lastEvent
            break
    
    durationMinutes = (endTime - startTime).total_seconds() / 60
    
    for i in range(len(group)) :
        firstEventList.append(firstProcedure)
        lastEventList.append(lastProcedure)
        startTimeList.append(startTime)
        endTimeList.append(endTime)
        timeList.append(durationMinutes)
        
cleanAnesthesiaEventDF['FirstEvent'] = firstEventList
cleanAnesthesiaEventDF['LastEvent'] = lastEventList
cleanAnesthesiaEventDF['StartOfProcedure'] = startTimeList
cleanAnesthesiaEventDF['EndOfProcedure'] = endTimeList
cleanAnesthesiaEventDF['TimeInMinutes'] = timeList
#cleanAnesthesiaEventDF = pd.concat([cleanAnesthesiaEventDF, pd.DataFrame(time)])


In [ ]:
for i in range(len(cleanAnesthesiaEventDF)) :
    if cleanAnesthesiaEventDF['Event'][i] == 'Induktion' and cleanAnesthesiaEventDF['EventTime'][i] != cleanAnesthesiaEventDF['StartOfProcedure'][i] :
        print(cleanAnesthesiaEventDF['CPR'][i])

In [ ]:
cleanAnesthesiaEventDF

In [ ]:
print(cleanAnesthesiaEventDF.value_counts('TimeInMinutes'))

Saving the procedure dataframe to csv

In [ ]:
cleanAnesthesiaEventDF.to_csv(r'FilteredData\CleanAnesthesiaTime.csv' , index = False)

In [ ]:
bins = [-float('inf'), 0, 1, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, float('inf')]

bin_labels = ['-','0', '1-100', '101-200', '201-300', '301-400', '401-500', '501-600', 
              '601-700', '701-800', '801-900', '901-1000', '1001-2000', '2001-3000', '3000+']


plt.figure(figsize=(12, 6))
hist = pd.cut(cleanAnesthesiaEventDF['TimeInMinutes'], bins=bins, labels=bin_labels, include_lowest=True).value_counts().sort_index().plot(kind='bar')


plt.xlabel('Procedure Time Range')
plt.ylabel('Frequency')
plt.title('Distribution of Procedure Times by Range')
plt.xticks(rotation=45) 
plt.show()

## Anesthesia data dataset

In [ ]:
anesthesiaDataDF = pd.read_csv(r"DataCSV\Anæstesidata.csv", sep = ';')
anesthesiaDataDF['Målingsdatotid'] = pd.to_datetime(anesthesiaDataDF['Målingsdatotid'])
anesthesiaDataDF

In [ ]:
len(list(set(cleanAnesthesiaEventDF['CPR'])))

In [ ]:
anesthesiaDataDF = anesthesiaDataDF[anesthesiaDataDF['CPR'].isin(cleanAnesthesiaEventDF['CPR'])]


In [ ]:
len(list(set(anesthesiaDataDF['CPR'])))

In [ ]:
cleanAnesthesiaEventPivotDF = cleanAnesthesiaEventDF.pivot(index='CPR', columns='Event', values='EventTime').reset_index()
anesthesiaDataMergedDF = pd.merge(anesthesiaDataDF, cleanAnesthesiaEventPivotDF, on='CPR')

In [ ]:
""" anesthesiaDataMergedDF['Saturation'] = np.where(anesthesiaDataMergedDF['Saturation'].isna(), anesthesiaDataMergedDF['Saturation'],  # If 'col1' is NaN, take 'col2'
                      np.where(anesthesiaDataMergedDF['SpO2'].isna(), anesthesiaDataMergedDF['Saturation'],  # If 'col2' is NaN, keep 'col1'
                               np.maximum(anesthesiaDataMergedDF['Saturation'], anesthesiaDataMergedDF['SpO2']))) """
anesthesiaDataMergedDF['Saturation'] = np.where(anesthesiaDataMergedDF['Saturation'].isna(), anesthesiaDataMergedDF['SpO2'], np.maximum(anesthesiaDataMergedDF['Saturation'], anesthesiaDataMergedDF['SpO2']))
anesthesiaDataMergedDF['Saturation'] = anesthesiaDataMergedDF['Saturation'].drop(columns = ['SpO2'])


In [ ]:
filter_condition = (anesthesiaDataMergedDF[['ETCO2', 'Saturation', 'Perf. Index']] > 0) & (anesthesiaDataMergedDF[['ETCO2', 'Saturation', 'Perf. Index']] < 300)

# Replace values not between 0 and 300 with NaN
anesthesiaDataMergedDF[['ETCO2', 'Saturation', 'Perf. Index']] = anesthesiaDataMergedDF[['ETCO2', 'Saturation', 'Perf. Index']].where(filter_condition)


In [ ]:
phase1 = anesthesiaDataMergedDF[(anesthesiaDataMergedDF['Målingsdatotid'] >= anesthesiaDataMergedDF['Induktion']) & (anesthesiaDataMergedDF['Målingsdatotid'] < anesthesiaDataMergedDF['Aorta tang på'])]
phase2 = anesthesiaDataMergedDF[(anesthesiaDataMergedDF['Målingsdatotid'] >= anesthesiaDataMergedDF['Aorta tang på']) & (anesthesiaDataMergedDF['Målingsdatotid'] < anesthesiaDataMergedDF['Aorta tang af'])]
phase3 = anesthesiaDataMergedDF[(anesthesiaDataMergedDF['Målingsdatotid'] >= anesthesiaDataMergedDF['Aorta tang af']) & (anesthesiaDataMergedDF['Målingsdatotid'] <= anesthesiaDataMergedDF['Stop Data Indsamling'])]
phase1 = phase1[['CPR' , 'ETCO2' , 'Saturation' , 'Perf. Index' ,'SpO2']]
phase2 = phase2[['CPR' , 'Saturation' , 'Perf. Index' ,'SpO2']]
# phase2 = phase2[['CPR' , 'ETCO2' , 'Saturation' , 'Perf. Index' ,'SpO2']]
phase3 = phase3[['CPR' , 'ETCO2' , 'Saturation' , 'Perf. Index' ,'SpO2']]

In [ ]:
percentiles = [5, 10, 25, 50, 75, 90, 95]
def calc_percentiles(group):
    return group.quantile([p/100 for p in percentiles]).values

In [ ]:
percentile_data = {}
for col in ['ETCO2', 'Saturation', 'Perf. Index']:
    percentile_data[col] = phase1.groupby('CPR')[col].apply(lambda x: pd.Series(calc_percentiles(x.dropna()), index=percentiles))
phase1DF = pd.concat(percentile_data, axis=1).unstack(level=1).reset_index()
phase1DF.columns = ['CPR'] + [f'{col}_p{p}' for col in ['ETCO2', 'Saturation', 'Perf. Index'] for p in percentiles]
phase1DF

In [ ]:
percentile_data = {}
for col in ['Saturation', 'Perf. Index']:
    percentile_data[col] = phase2.groupby('CPR')[col].apply(lambda x: pd.Series(calc_percentiles(x.dropna()), index=percentiles))
phase2DF = pd.concat(percentile_data, axis=1).unstack(level=1).reset_index()
phase2DF.columns = ['CPR'] + [f'{col}_p{p}' for col in ['Saturation', 'Perf. Index'] for p in percentiles]
phase2DF

In [ ]:
len(list(set(phase2.dropna()['CPR'])))

In [ ]:
percentile_data = {}
for col in ['ETCO2', 'Saturation', 'Perf. Index']:
    percentile_data[col] = phase3.groupby('CPR')[col].apply(lambda x: pd.Series(calc_percentiles(x.dropna()), index=percentiles))
phase3DF = pd.concat(percentile_data, axis=1).unstack(level=1).reset_index()
phase3DF.columns = ['CPR'] + [f'{col}_p{p}' for col in ['ETCO2', 'Saturation', 'Perf. Index'] for p in percentiles]
phase3DF

In [ ]:
phase1DF.to_csv(r'FilteredData/phase1Anesthesia.csv' , index = False)
phase2DF.to_csv(r'FilteredData/phase2Anesthesia.csv' , index = False)
phase3DF.to_csv(r'FilteredData/phase3Anesthesia.csv' , index = False)

## Preoperation conference 

Seems like this patient is missing from anesthesiaEvent

In [ ]:
#Had some problems reading the file because of the encoding, so chardet is used to find the encoding



file = r"DataCSV\konference_notat.csv"
with open(file, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

In [ ]:
preOpConferenceDF = pd.read_csv(file, sep = ';', encoding = 'ISO-8859-1')

preOpConferenceDFDic = {'CPR'              : 'CPR',
                        'Måling'           : 'Measurement',
                        'Måling ID'        : 'MeasurementID',
                        'Målingstidspunkt' : 'TimeOfMeasurement',
                        'Værdi'            : 'Value'}

preOpConferenceDF = preOpConferenceDF.rename(columns = preOpConferenceDFDic)

preOpConferenceDF = preOpConferenceDF[preOpConferenceDF['CPR'].isin(populationDF['CPR'])].reset_index(drop = True)

preOpConferenceDF

In [ ]:
weightCount = preOpConferenceDF.groupby('CPR').filter(lambda group: (group['Measurement'] == 'Vægt').any())['CPR'].nunique()
weightCount

In [ ]:
preOpConferenceDF['Measurement'].value_counts().to_csv(r'FilteredData/Conference Measurement Count.csv')

Creating table of each measurement with its associated count

In [ ]:
' TODO: need to do more work on preOpConference data before we can apply filter'
viewMeasurements = pd.DataFrame({'Measurement' : list(set(preOpConferenceDF['Measurement'])), 'Count' : list(set(preOpConferenceDF['Measurement']))})

viewMeasurements['Count'] = viewMeasurements['Measurement'].map(preOpConferenceDF['Measurement'].value_counts())
viewMeasurements = viewMeasurements.sort_values('Count' , ascending = False)
viewMeasurements.to_csv(r'FilteredData\UniqueConferenceMeasurements.csv', index = False)

In [ ]:
preOpConferenceDF['Measurement'].unique()

Checking the unique measurement types

In [ ]:
len(list(set(preOpConferenceDF['Measurement'])))

In [ ]:
preOpConferenceDF.count()

## Alcohol and smoking data

Need to do some heavy work to make this work since we don't have this data timed

In [ ]:
alcAndSmokeDF = pd.read_csv(r'DataCSV\Alkohol og rygning.csv', sep = ';')

alcAndSmokeDic = {'CPR'                  : 'CPR',
                  'Genstande per uge'    : 'AlcoholPerWeek',
                  'Rygning'              : 'Smoking'}

alcAndSmokeDF = alcAndSmokeDF.rename(columns = alcAndSmokeDic)

alcAndSmokeDF = alcAndSmokeDF[alcAndSmokeDF['CPR'].isin(populationDF['CPR'])].reset_index(drop = True)

alcAndSmokeDF

In [ ]:
alcAndSmokeDF.count()

In [ ]:
alcAndSmokeDF['Smoking'].value_counts()

In [ ]:
alcPerWeek = alcAndSmokeDF['AlcoholPerWeek'].value_counts()

In [ ]:
for row in range(len(alcAndSmokeDF)) : 
    if isinstance(alcAndSmokeDF['AlcoholPerWeek'][row], str) and '-' in alcAndSmokeDF['AlcoholPerWeek'][row] :
        alcPerWeek = alcAndSmokeDF['AlcoholPerWeek'][row].split('-')
        totalAlcoholPerWeek = 0
        for i in range(len(alcPerWeek)) :
            totalAlcoholPerWeek += int(alcPerWeek[i])
        alcAndSmokeDF.loc[row, 'AlcoholPerWeek'] = totalAlcoholPerWeek/len(alcPerWeek)
#alcAndSmokeDF['AlcoholPerWeek'] = alcAndSmokeDF['AlcoholPerWeek'].fillna(0).astype(int)

In [ ]:
''' 
Aldrig vurderet = NaN
Ukendt = NaN
Aldrig = 0
Udsat for passiv rygning - aldrig været ryger = 0
Lejlighedsvis ryger = 1
Nogle dage = 1
Hver dag = 1
Ryger, aktuel status ukendt = 1
Storryger = 1
Tidligere = 2
''';

dict = {'Aldrig vurderet'                               : -1,
        'Ukendt'                                        : -1,
        'Aldrig'                                        : 0,
        'Udsat for passiv rygning - aldrig været ryger' : 0,
        'Lejlighedsvis ryger'                           : 1,
        'Nogle dage'                                    : 1,
        'Hver dag'                                      : 1,
        'Ryger, aktuel status ukendt'                   : 1,
        'Storryger'                                     : 1,
        'Tidligere'                                     : 2
        }
alcAndSmokeDF = alcAndSmokeDF.replace({'Smoking' : dict})
alcAndSmokeDF

## Blood samples

In [ ]:
bloodSamplesDF = pd.read_csv(r'DataCSV\Lab svar.csv', sep = ';')
bloodSamplesDF


In [ ]:
bloodSamplesNames = [ 'CPR', 'OrderName', 'ResultTime', 'ResultValue']
bloodSamples2ndDF = pd.read_csv(r'DataCSV\INC4924474 - Labsvar.csv', sep = ';', names = bloodSamplesNames, header = None)
bloodSamples2ndDF

In [ ]:
bloodSamples2ndDF = bloodSamples2ndDF[bloodSamples2ndDF['CPR'].isin(PATSdf['CPR'])].reset_index(drop = True)

In [ ]:
len(list(set(bloodSamples2ndDF[bloodSamples2ndDF['CPR'].isin(PATSdf['CPR'])])))

In [ ]:

bloodSamplesDic = {'cpr'              : 'CPR',
                   'Best./ord. navn'  : 'OrderName',
                   'Resultatdatotid'  : 'ResultTime',
                   'Resultatværdi'    : 'ResultValue'}

bloodSamplesDF = bloodSamplesDF.rename(columns = bloodSamplesDic)

bloodSamplesDF = bloodSamplesDF[bloodSamplesDF['CPR'].isin(PATSdf['CPR'])].reset_index(drop = True)

bloodSamplesDF

In [ ]:
bloodSamplesDF = pd.concat([bloodSamples2ndDF, bloodSamplesDF], ignore_index=True)
bloodSamplesDF = bloodSamplesDF.sort_values('CPR')
bloodSamplesDF

In [ ]:
len(list(set(bloodSamplesDF['CPR'])))

In [ ]:
bloodSamplesDF.count()

In [ ]:
bloodSamplesDF[bloodSamplesDF.isnull().any(axis=1)]

In [ ]:
bloodSamplesDF = bloodSamplesDF.dropna()
bloodSamplesDF = bloodSamplesDF[bloodSamplesDF.ResultValue != 'Ej målt']

In [ ]:
len(list(set(bloodSamplesDF['CPR'])))

In [ ]:
viewBloodResults = pd.DataFrame({'ResultValue' : list(set(bloodSamplesDF['ResultValue'])), 'Count' : list(set(bloodSamplesDF['ResultValue']))})


viewBloodResults['Count'] = viewBloodResults['ResultValue'].map(bloodSamplesDF['ResultValue'].value_counts())
viewBloodResults = viewBloodResults.sort_values('Count' , ascending = False)

In [ ]:
viewBloodOrders = pd.DataFrame({'OrderName' : list(set(bloodSamplesDF['OrderName'])), 'Count' : list(set(bloodSamplesDF['OrderName']))})


viewBloodOrders['Count'] = viewBloodOrders['OrderName'].map(bloodSamplesDF['OrderName'].value_counts())
viewBloodOrders = viewBloodOrders.sort_values('Count' , ascending = False)
#viewBloodOrders.to_csv(r'FilteredData\BloodSampleOrder.csv', index = False)

## Diagnoses of patients

In [ ]:
diagnosesDF = pd.read_csv(r'DataCSV\Problemliste.csv', sep = ';')

diagnosesDic = {'CPR'           : 'CPR',
                'Dato'          : 'Date',
                'Diagnose SKS'  : 'DiagnosisSKS',
                'Diagnose navn' : 'DiagnosisName'}

diagnosesDF = diagnosesDF.rename(columns = diagnosesDic)

diagnosesDF = diagnosesDF[diagnosesDF['CPR'].isin(populationDF['CPR'])].reset_index(drop = True)

diagnosesDF

In [ ]:
diagnosesDF.count()

## Vitals dataset

In [ ]:
vitalsDF = pd.read_csv(r'DataCSV\cirk. vitale værdier.csv', sep = ';')

vitalsDF

In [ ]:

vitalsDic = {'CPR'                          : 'CPR',
            'Målingsdatotid'                : 'TimeOfMeasurement',
            'Saturation'	                : 'BloodSaturation',
            'Blodtryk'	                    : 'BloodPressure',
            'MAP'                           : 'BloodPressureMean',
            'Invasivt BT - ABP (sys/dia)'   : 'ArterialBloodPressure',
            'Invasivt BT - ABP (mean)'      : 'ArterialBPMean',
            'CVP (mean)'                    : 'CentralVenousPressureMean',
            'Respirationsfrekvens'          : 'RespirationFreq',
            'Blæretemperatur'               : 'TemperatureBladder',
            'Temp.'                         : 'Temperature',
            'Blodtemperatur'                : 'TemperatureInFahrenheit',
            'Temp (in Celsius)'             : 'TemperatureInCelsius',
            'Puls (fra SAT-måler)'          : 'PulseSAT',
            'Puls'                          : 'Pulse',
            'PAP (middel)'	                : 'PAPMiddle',
            'PAP (sys/dia)'                 : 'PAPsysDia'}

vitalsDF = vitalsDF.rename(columns = vitalsDic)

vitalsDF = vitalsDF[vitalsDF['CPR'].isin(populationDF['CPR'])].reset_index(drop = True)

vitalsDF

In [ ]:
len(list(set(vitalsDF['CPR'])))

In [ ]:
vitalsDF.count()

### Merging columns

In [ ]:
vitalsDF['TemperatureInFahrenheit'] = (vitalsDF['TemperatureInFahrenheit']-32)*5/9
vitalsDF['TemperatureInFahrenheit'] = vitalsDF['TemperatureInFahrenheit'].round(1)
vitalsDF['TemperatureInCelsius'] = vitalsDF['TemperatureInCelsius'].fillna(vitalsDF['TemperatureInFahrenheit'])
vitalsDF['TemperatureInCelsius']


In [ ]:
vitalsDF['PulseSAT'] = vitalsDF['PulseSAT'].fillna(vitalsDF['Pulse'])
vitalsDF['Pulse'] = vitalsDF['PulseSAT']

In [ ]:
vitalsDF['BloodPressure'] = vitalsDF['BloodPressure'].fillna(vitalsDF['ArterialBloodPressure'])
vitalsDF['ArterialBloodPressure'] = vitalsDF['BloodPressure']

In [ ]:
vitalsDF['BloodPressureMean'] = vitalsDF['BloodPressureMean'].fillna(vitalsDF['ArterialBPMean'])
vitalsDF['ArterialBPMean'] = vitalsDF['BloodPressureMean']

In [ ]:
for i in range(len(vitalsDF)) :
    if pd.notna(vitalsDF['ArterialBloodPressure'][i]) and pd.isna(vitalsDF['ArterialBPMean'][i]):
        numbers = vitalsDF['ArterialBloodPressure'][i].split('/')
        if not int(numbers[1]) - int(numbers[0]) == 0 :
            vitalsDF.loc[i, 'ArterialBPMean'] = round(int(numbers[1]) + (int(numbers[0]) - int(numbers[1]))/3)
        else :
            vitalsDF.loc[i, 'ArterialBPMean'] = 0


In [ ]:
for i in range(len(vitalsDF)) :
    if pd.notna(vitalsDF['PAPsysDia'][i]) and pd.isna(vitalsDF['PAPMiddle'][i]):
        numbers = vitalsDF['PAPsysDia'][i].split('/')
        if not int(numbers[1]) - int(numbers[0]) == 0 :
            vitalsDF.loc[i, 'PAPMiddle'] = round(int(numbers[1]) + (int(numbers[0]) - int(numbers[1]))/3)
        else :
            vitalsDF.loc[i, 'PAPMiddle'] = 0

In [ ]:
vitalsDF = vitalsDF.drop(columns = ['RespirationFreq' , 'TemperatureBladder' , 'Temperature' , 'TemperatureInFahrenheit', 
                                    'PulseSAT', 'BloodPressure', 'BloodPressureMean' , 'ArterialBloodPressure' , 'PAPsysDia'])
vitalsDF.count()

In [ ]:
dateFormat1 = r'%Y-%m-%d %H:%M:%S.000'
vitalsDF['TimeOfMeasurement'] = pd.to_datetime(vitalsDF['TimeOfMeasurement'], format = dateFormat1)

In [ ]:
for cpr in cleanAnesthesiaEventDF['CPR'].unique() :

    patientAnesthesia = cleanAnesthesiaEventDF[cleanAnesthesiaEventDF['CPR'] == cpr]

    patientVitals = vitalsDF[vitalsDF['CPR'] == cpr]
    #print(patientAnesthesia)
    induktionTime = patientAnesthesia[patientAnesthesia['Event'] == 'Induktion']['EventTime'].reset_index(drop = True)
    clampOnTime = patientAnesthesia[patientAnesthesia['Event'] == 'Aorta tang på']['EventTime'].reset_index(drop = True)
    clampOffTime = patientAnesthesia[patientAnesthesia['Event'] == 'Aorta tang af']['EventTime'].reset_index(drop = True)
    stopDataTime = patientAnesthesia[patientAnesthesia['Event'] == 'Stop Data Indsamling']['EventTime'].reset_index(drop = True)
    patientInduktionTimes = patientVitals[(patientVitals['TimeOfMeasurement'] > induktionTime.iloc[0]) & (patientVitals['TimeOfMeasurement'] < clampOnTime.iloc[0])]['TimeOfMeasurement']
    #print(induktionTime)
    print(len(patientInduktionTimes))
    break



## Hemodynamic data

In [ ]:
hemodynamicDF = pd.read_csv(r'FilteredData\HemodynamicMeasuresRAW.csv')
hemodynamicDF

In [ ]:
hemodynamicBP = hemodynamicDF[(hemodynamicDF['Målingsnavn'].str[:3] == 'PAP') | (hemodynamicDF['Målingsnavn'].str[:3] == 'LAP') | (hemodynamicDF['Målingsnavn'].str[:3] == 'MAP')]

In [ ]:
hemodynamicBP

In [ ]:
hemodynamicBP['Målingsnavn'].value_counts()

In [ ]:
def sysDiaSplit(value) :
    if isinstance(value, str) and '/' in value:
        sys, dia = value.split('/')
        print(round(int(dia)/3 + 2*int(sys)/3))
        if False :
            return round(int(dia)/3 + 2*int(sys)/3)
    #return value

hemodynamicBP['Værdi'] = hemodynamicBP.apply(lambda row: sysDiaSplit(row['Værdi']) if 'PAP' in row['Målingsnavn'] else row['Værdi'], axis=1)

hemodynamicBP['Målingsnavn'] = hemodynamicBP['Målingsnavn'].replace(['PAP', 'PAP (sys/dia)', 'PAP (middel)'], 'PAP (mean)')

hemodynamicBP['Værdi'] = hemodynamicBP.apply(lambda row: sysDiaSplit(row['Værdi']) if 'PAP' in row['Målingsnavn'] else row['Værdi'], axis=1)

hemodynamicBP['Målingsnavn'] = hemodynamicBP['Målingsnavn'].replace(['LAP', 'LAP (sys/dia)', 'LAP (middel)'], 'LAP (mean)')

hemodynamicBP['Målingsnavn'].value_counts()

In [ ]:
hemodynamicMeasures = hemodynamicDF['Målingsnavn'].value_counts()

In [ ]:
hemodynamicBP.to_csv(r'FilteredData\HemodynamicMeasuresCount.csv')

## Ekkofiler

### Ekkokardiografi.csv

In [ ]:
""" echocardiographyDF = pd.read_csv(r'DataCSV\Ekkokardiografi.csv', sep = ';') """

In [ ]:
""" echocardiographyDF = echocardiographyDF[echocardiographyDF['CPR'].isin(populationDF['CPR'])]
len(list(set(echocardiographyDF['CPR']))) """

In [ ]:
""" echocardiographyDF """

In [ ]:
""" echocardiographyDF.count() """

### Viewpoint.csv

In [ ]:
""" viewpointDF = pd.read_csv(r'DataCSV\viewpoint.csv')
viewpointDF = viewpointDF[viewpointDF['CPR.nummer']]
viewpointDF """

In [ ]:
""" len(list(set(viewpointDF['CPR.nummer']))) """

In [ ]:
""" tester = viewpointDF[viewpointDF['FD'] == '30-12-1899']

len(list(set(tester['CPR.nummer']))) """

## Saving the new data

### Population

In [ ]:
PATSdf = PATSdf[PATSdf['CPR'].isin(anesthesiaEventDF['CPR'])]
populationDF = populationDF[populationDF['CPR'].isin(PATSdf['CPR'])]
len(list(set(populationDF['CPR'])))

In [ ]:
populationDF.to_csv(r'FilteredData\Population.csv', index = False)

### SKS filter

In [ ]:
SKSFilter.to_csv(r'FilteredData\SKS Valid.csv' , index = False)
SKSfilteredOut.to_csv(r'FilteredData\SKS Invalid.csv' , index = False)

### PATS

In [ ]:
len(list(set(PATSdf['CPR'])))

In [ ]:
PATSdf.to_csv(r'FilteredData\PATS.csv')

### Anesthesia data

In [ ]:
anesthesiaEventDF.to_csv(r'FilteredData\Anesthesia Events.csv')

### Preoperation conference data

In [ ]:
preOpConferenceDF.to_csv(r'FilteredData\Preoperation Conference.csv', index = False)

### Alcohol and smoking

In [ ]:
alcAndSmokeDF.to_csv(r'FilteredData\Alcohol and Smoking.csv')

### Blood samples

In [ ]:
bloodSamplesDF.to_csv(r'FilteredData\Blood Samples.csv', index = False)

### Diagnoses of patients

In [ ]:
diagnosesDF.to_csv(r'FilteredData\Diagnoses of Patients.csv' , index = False)

### Vitals

In [ ]:
vitalsDF.to_csv(r'FilteredData\Vitals.csv', index = False)

### Anonymous versions of all the files

In [ ]:
vitalsCensored = vitalsDF.drop(columns = ['CPR'])
diagnosesCensored = diagnosesDF.drop(columns = ['CPR'])
bloodSamplesCensored = bloodSamplesDF.drop(columns = ['CPR'])
alcAndSmokeCensored = alcAndSmokeDF.drop(columns = ['CPR'])
preOpConferenceCensored = preOpConferenceDF.drop(columns = ['CPR'])
anesthesiaEventCensored = anesthesiaEventDF.drop(columns = ['CPR'])
PATSCensored = PATSdf.drop(columns = ['CPR'])
populationCensored = populationDF.drop(columns = ['CPR'])
hemodynamicBPCensored = hemodynamicBP.drop(columns = ['cpr'])
cleanAnesthesiaEventCensored = cleanAnesthesiaEventDF.drop(columns = ['CPR'])

In [ ]:
vitalsCensored.to_csv(r'DataAnonymous\Vitals.csv')
diagnosesCensored.to_csv(r'DataAnonymous\Diagnoses of Patients.csv')
bloodSamplesCensored.to_csv(r'DataAnonymous\Blood Samples.csv')
alcAndSmokeCensored.to_csv(r'DataAnonymous\Alcohol and smoking.csv')
preOpConferenceCensored.to_csv(r'DataAnonymous\Conference data.csv')
#anesthesiaEventCensored.to_csv(r'DataAnonymous\Anesthesia Events.csv')
PATSCensored.to_csv(r'DataAnonymous\PATS.csv')
populationCensored.to_csv(r'DataAnonymous\Population.csv')
hemodynamicBPCensored.to_csv(r'DataAnonymous\Hemodynamic Blood Pressure.csv')
cleanAnesthesiaEventCensored.to_csv(r'DataAnonymous\Clean anesthesia.csv')